# [第10章: ベクトル空間法 (II)](http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch10)
第10章では，前章に引き続き単語ベクトルの学習に取り組む．

## 90. word2vecによる学習
81で作成したコーパスに対して[word2vec](https://code.google.com/p/word2vec/)を適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [1]:
!ls Output/Chapter9/81.txt

Output/Chapter9/81.txt


In [22]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm

In [23]:
%%time
model = None

with open("Output/Chapter9/81.txt") as f:
    sentences = [line.replace("\n", "").split() for line in tqdm(f)]
    model = Word2Vec(sentences=sentences, size=300, min_count=10, seed=90)
    model.save("Output/Chapter10/90_word2vec.model")

284434it [00:02, 111495.53it/s]


CPU times: user 2min 49s, sys: 1.01 s, total: 2min 50s
Wall time: 1min 8s


In [24]:
model = Word2Vec.load("Output/Chapter10/90_word2vec.model")

In [29]:
# 86
model.wv["United_States"].shape

(300,)

In [30]:
# 86
model.wv["United_States"]

array([-0.44872293,  1.6334685 ,  0.42069754,  0.01393384, -0.16998678,
        0.02382363,  0.4402721 , -0.0651859 , -0.376896  ,  0.84776735,
       -0.4474914 , -0.71033776, -0.40516818, -0.26205367,  0.5669195 ,
       -0.8465921 , -0.81093657, -0.3613166 , -0.33018088,  1.2646618 ,
        0.08996649,  1.3481137 , -0.0038776 , -0.17100886,  0.3017421 ,
        0.01281976, -0.54665107, -0.908616  ,  0.60607755,  0.9918977 ,
        1.0866829 ,  0.19219446, -0.9581521 ,  0.3187664 , -1.1826046 ,
       -0.513105  , -0.84238786,  0.1298967 ,  0.5220589 ,  0.14251645,
       -0.4307814 ,  1.4381793 , -1.3689756 ,  1.2477808 ,  1.2637137 ,
        0.5176013 , -0.744362  ,  0.03129834,  1.1492134 ,  0.5294556 ,
        0.21340181,  0.7044161 , -0.76726604,  1.4851089 ,  0.5223592 ,
        0.729374  , -0.16693877,  0.6762518 ,  2.0744722 ,  1.2172992 ,
       -0.6718626 , -0.30356175,  0.38613924,  0.17126812, -0.10497062,
        0.6619575 , -0.38637236,  0.18190609, -0.43362013,  0.02

In [32]:
# 87
model.wv.n_similarity(["United_States"], ["U.S"])

0.83274084

In [35]:
# 88
for i, (word, cos_sim) in enumerate(model.wv.similar_by_word("England", topn=10)):
    args = (i+1, word, cos_sim)
    print("[%d/10] %s : %f" % args)

[1/10] Scotland : 0.777856
[2/10] Wales : 0.739699
[3/10] Ireland : 0.613046
[4/10] Britain : 0.608234
[5/10] London : 0.574067
[6/10] Sweden : 0.563965
[7/10] Plymouth : 0.559828
[8/10] Birmingham : 0.554329
[9/10] Newcastle : 0.548365
[10/10] Hampshire : 0.545805


In [36]:
# 89
x_greece = model.wv["Spain"] - model.wv["Madrid"] + model.wv["Athens"]
for i, (word, cos_sim) in enumerate(model.wv.similar_by_vector(x_greece, topn=10)):
    args = (i+1, word, cos_sim)
    print("[%d/10] %s : %f" % args)

[1/10] Spain : 0.885322
[2/10] Italy : 0.801836
[3/10] Austria : 0.765410
[4/10] Denmark : 0.765014
[5/10] Egypt : 0.761510
[6/10] Russia : 0.761072
[7/10] Sweden : 0.746197
[8/10] Belgium : 0.743078
[9/10] Norway : 0.739321
[10/10] Portugal : 0.736502


## 91. アナロジーデータの準備
[単語アナロジーの評価データ](https://raw.githubusercontent.com/svn2github/word2vec/master/questions-words.txt)をダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

## 92. アナロジーデータへの適用
91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

## 93. アナロジータスクの正解率の計算
92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

## 94. WordSimilarity-353での類似度計算
[The WordSimilarity-353 Test Collection](http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/)の評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

## 95. WordSimilarity-353での評価
94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

## 96. 国名に関するベクトルの抽出
word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

## 97. k-meansクラスタリング
96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5として実行せよ．

## 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

## 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．